In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date
import os, platform

In [4]:
path = "..\\..\\_data\\sbs\\B_RawData\\bancos" # Relative path

if 'linux' not in platform.system() or 'Darwin' not in platform.system():
    path = "../../_data/sbs/B_RawData/bancos"

all_files = [f for f in os.listdir(path) if f[-4:] == 'xlsx']

In [5]:
def get_date(file):
    str_date = '_'.join(file[0:-5].split('_')[2:5])
    return datetime.strptime(str_date, '%d_%m_%Y').date()

## SOLUCIÓN PREGUNTA 1
Importamos la data según sus distintos formatos

### Format 1
Including data between 2002 to April 2008

In [6]:
format_1_files = [
    file for file in all_files 
    if date(2008, 4, 30) > get_date(file) > date(2002, 9, 30)
]

def sanitize_format_1(file):
    df = pd.read_excel(f'{path}/{file}', sheet_name="Sheet1", header=1, skiprows=1)
    df = df.rename(
        columns={
            'Unnamed: 0': 'bank_name',
            'Tarjetas de crédito': 'credit_card',
            'HIPOTECARIO': 'mortgage'
        }
    )
    df['date_info'] = get_date(file)
    return df

format_1_df = pd.concat([sanitize_format_1(file) for file in format_1_files], ignore_index=True)[['bank_name', 'credit_card', 'mortgage', 'date_info']]

### Format 2
Including data between May 2008 to June 2010

In [7]:
format_2_files = [
    file for file in all_files 
    if date(2008, 5, 30) < get_date(file) < date(2010, 6, 1)
]

def sanitize_format_2(file):
    first_table = pd.read_excel(f'{path}/{file}', sheet_name="Sheet1", header=1, usecols="A:V").drop(0)
    first_table = first_table.rename(columns={'Tasa Anual (%)': 'bank_name'})
    second_table = pd.read_excel(f'{path}/{file}', sheet_name="Sheet1", skiprows= range(0, 3), usecols="W:AK")
    second_table = second_table.rename(
        columns={
            'Promedio': 'credit_card',
            'PRÉSTAMOS HIPOTECARIOS': 'mortgage'
        }
    )
    second_table['date_info'] = get_date(file)
    return first_table.merge(second_table, left_index=True, right_index=True)

format_2_df = pd.concat([sanitize_format_2(file) for file in format_2_files], ignore_index=True)[['bank_name', 'credit_card', 'mortgage', 'date_info']]

### Format 3
Including data between Sept 2010 - 2015

In [8]:
format_3_files = [
    file for file in all_files
    if date(2010, 9, 10) < get_date(file) < date(2015, 12, 31)
]

def sanitize_format_3(file):
    df = pd.read_excel(f'{path}/{file}', sheet_name="Sheet1")
    df.set_index('Tasa Anual (%)', inplace=True)
    df = df.loc[~df.index.duplicated(keep='first')].T.iloc[:, :-2]
    df['date_info'] = get_date(file) # Add date info
    df = df.reset_index().rename_axis(None, axis=1).rename(
        columns={
            'index': 'bank_name',
            'Tarjetas de Crédito': 'credit_card',
            'Hipotecarios': 'mortgage'
        })
    df['bank_name'] = df['bank_name'].str.upper() # Clean bank names
    return df
format_3_df = pd.concat([sanitize_format_3(file) for file in format_3_files])[['bank_name', 'credit_card', 'mortgage', 'date_info']]

### Format 4
Including data between 2016 to 2022 and 2001

In [9]:
format_4_files = [
    file for file in all_files
    if (date(2016, 1, 1) < get_date(file) < date(2022, 12, 31)) or get_date(file).year == 2001
]

def sanitize_format_4(file):
    df = pd.read_excel(f'{path}/{file}', sheet_name="Sheet1")
    df.set_index('Tasa Anual (%)', inplace=True)
    df = df.loc[~df.index.duplicated(keep='first')].T
    df['date_info'] = get_date(file)
    df = df.reset_index().rename_axis(None, axis=1).rename(
        columns={
            'index': 'bank_name',
            'Tarjetas de Crédito': 'credit_card',
            'Préstamos hipotecarios para vivienda': 'mortgage'
        }
    )
    return df

format_4_df = pd.concat([sanitize_format_4(file) for file in format_4_files])[['bank_name', 'credit_card', 'mortgage', 'date_info']]

### Format 5
Including data between Jul - Aug 2010

In [10]:
format_5_files = [
    file for file in all_files
    if (7 <= get_date(file).month <= 8) and get_date(file).year == 2010
]

def sanitize_format_5(file):
    first_table = pd.read_excel(f'{path}/{file}', sheet_name="Sheet1", usecols="A:H", skiprows=[1, 2])
    first_table = first_table.rename(columns={'Tasa Anual (%)': 'bank_name'})
    second_table = pd.read_excel(f'{path}/{file}', sheet_name="Sheet1", skiprows= [1, 2, 3], usecols="U:AF")
    second_table = second_table.rename(
        columns={
            'CRÉDITOS DE CONSUMO.3': 'credit_card',
            'PRÉSTAMOS HIPOTECARIOS': 'mortgage'
        }
    )
    second_table['date_info'] = get_date(file)
    return first_table.merge(second_table, left_index=True, right_index=True)

format_5_df = pd.concat([sanitize_format_5(file) for file in format_5_files])[['bank_name', 'credit_card', 'mortgage', 'date_info']]

## SOLUCIÓN PREGUNTA 2 
Unimos todas las bases de datos

In [11]:
db = pd.concat(
    [format_1_df,
    format_2_df,
    format_3_df,
    format_4_df,
    format_5_df],
    axis=0
).reset_index(drop=True)

## Cleaning Numeric Values

In [12]:
db['credit_card'] = pd.to_numeric(db['credit_card'], errors='coerce')
db['mortgage'] = pd.to_numeric(db['mortgage'], errors='coerce')
db = db.replace(np.nan, 0, regex=True)

## Cleaning Banks names

In [13]:
db['bank_name'] = db['bank_name'].str.upper()

In [14]:
db['bank_name'] = db['bank_name']\
    .str\
    .replace('BANCO DE', '')\
    .replace('BANCO', '')\
    .replace('DEL PERU', '')\
    .replace('DE PERU', '')\
    .replace('PERU', '')\
    .replace('FINANCIERO', 'PICHINCHA')\
    .replace('FINANCIERO  *', 'PICHINCHA')\
    .replace('AZTECA', 'ALFIN') \
    .replace('AZTECA  *', 'ALFIN') \
    .replace('HSBC(*)', 'HSBC')\
    .replace('CONTINENTAL', 'BBVA')\
    .replace('BANBIF', 'BIF')\
    .replace('CREDITO', 'CRÉDITO')\
    .replace('BNP PARIBAS EL', 'BNP')\
    .replace('DEUTSCHE BANK PERU', 'DEUTSCHE')\
    .replace('B SANTANDER CENTRAL', 'SANTANDER') \
    .replace('SANTANDER PERU', 'SANTANDER') \
    .replace('SCOTIABANK PERU', 'SCOTIABANK')\
    .replace('MIBANCO', 'MI')

In [15]:
db = db[db['bank_name'].str.contains("PROMEDIO") == False]

In [16]:
db

,bank_name,credit_card,mortgage,date_info
0,TRABAJO,72.54,0.00,2004-02-26
1,BBVA,48.57,15.92,2004-02-26
2,COMERCIO,24.01,0.00,2004-02-26
3,CREDITO,34.84,0.00,2004-02-26
4,BANCO PICHINCHA,51.11,0.00,2004-02-26
...,...,...,...,...
3749,BANCO FALABELLA,0.00,0.00,2010-08-31
3750,SANTANDER,0.00,0.00,2010-08-31
3751,BANCO RIPLEY,0.00,0.00,2010-08-31
3752,ALFIN BANCO,0.00,0.00,2010-08-31


## SOLUCIÓN PREGUNTA 3
Identificamos los 5 bancos con tasas hipotecarias y crédito de consumo más caras

In [18]:
#Creamos una columna "year" generada a partir de date_info, se está empleado "%y" como formato para expresar el año mediante la función strftime

db["year"] = [*map(lambda x: x.strftime("%Y"), db["date_info"])]

In [40]:
# Generamos tabla con 5 bancos con mayor tasa de préstamo hipotecario en cada año
top5_hipo = db.groupby('year').apply(lambda x: x.nlargest(5, 'mortgage')).reset_index(drop=True)
top5_hipo = top5_hipo.drop(["date_info","credit_card"], axis=1).reset_index(drop=True)
top5_hipo.rename(
    columns={"bank_name": "banks", "mortgage": "rate_value"},
    inplace=True,
)
top5_hipo["rate_concept"] = "Préstamo hipotecario" 

In [41]:
# Generamos tabla con 5 bancos con mayor tasa de crédito de consumo en cada año
top5_credi = db.groupby('year').apply(lambda x: x.nlargest(5, 'credit_card')).reset_index(drop=True)
top5_credi = top5_credi.drop(["date_info","mortgage"], axis=1).reset_index(drop=True)
top5_credi.rename(
    columns={"bank_name": "banks", "credit_card": "rate_value"},
    inplace=True,
)
top5_credi["rate_concept"] = "Crédito de consumo" 

## SOLUCIÓN PREGUNTA 4
Exportamos un excel por cada banco

In [47]:
#Nuevo path relativo
path = "..\\..\\_Assigment_4\\output\\sbs" # Relative path

In [51]:
db = db.sort_values("date_info")

In [52]:
# verifica si existen esas 3 carpetas, en caso no existan, las crea

if not os.path.exists("output"):
    os.makedirs("output")
if not os.path.exists("output/sbs"):
    os.makedirs("output/sbs")
if not os.path.exists("output/sbs/group8"):
    os.makedirs("output/sbs/group8")

In [53]:
#dentro de base db, iteramos sobre la columna banks, seleccionando los elementos que coincidan entre si (Valores únicos), creando un subconjunto
#dicho subconjunto se guarda dentro de un excel en la ruta indicada (output/sbs/group8)

for x in db.bank_name.unique().tolist():
    db.loc[db["bank_name"] == x,:].to_excel("output/sbs/group8/table_"+x.lower()+".xlsx")

# INTEGRANTES DE ESTA TAREA:
- Gabriela Isabel Calvo Portocarrero
- Angel Mauricio Ibañez Abanto
- Juan Diego Meléndez Aponte